In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
!pip install trimesh --upgrade
!pip install open3d --upgrade
!pip install natsort

In [3]:
from typing import List, Tuple
from natsort import natsorted
from tqdm import tqdm

import os
import json
import errno
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as IPd

import plotly.express as px
import plotly.graph_objs as go

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import trimesh
import open3d as o3d

from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

from sklearn.svm import SVC

from path import Path
import scipy.spatial.distance
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
o3d.utility.random.seed(seed)
torch.manual_seed(seed)

In [5]:
BASE_PATH = '/kaggle/input/modelnet10-classification/modelnet10'

DATA_PATH   = os.path.join(BASE_PATH, 'dataset')
LABEL_PATH  = os.path.join(BASE_PATH, 'class2Label.json')
SUBMIT_PATH = os.path.join(BASE_PATH, 'sample_submit.csv')

In [6]:
class_name_list = sorted(os.listdir(os.path.join(DATA_PATH, 'train')))

scene_list = list()

### Add point cloud
for class_name in class_name_list:
    off = random.choice(os.listdir(os.path.join(DATA_PATH, 'train', class_name)))
    
    mesh = trimesh.load(os.path.join(DATA_PATH, 'train', class_name, off))
    
    scene = trimesh.Scene()
    
    scene.add_geometry(mesh)
    
    scene_list.append(scene)

In [75]:
### Read data
def read_off(off: str) -> Tuple[np.array, np.array]:
    with open(off, 'r') as f:
        lines = f.readlines()
        lines_1 = lines[0].strip()
        if lines_1 != 'OFF':
            raise ValueError('Not a valid OFF header')

        n_vertex, n_face, n_edge = map(int, lines[1].strip().split())

        vertices = list()
        for i in range(n_vertex):
            vertex = list(map(float, lines[i + 2].strip().split()))
            vertices.append(vertex)

        faces = list()
        for i in range(n_face):
            face = list(map(int, lines[i + n_vertex + 2].strip().split()))
            faces.append(face[1:])
        
        return np.array(vertices), np.array(faces)

In [158]:
### Point Sampling
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        ### barycentric coordinates on a triangle
        ### https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points

In [159]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [160]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [168]:
def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

In [173]:
### Load Data
class ModelNet10(Dataset):
    def __init__(self, root: str, split: str, class2label:dict, transform=default_transforms()):
        self.root = root
        self.split = split.lower()
        self.transforms = transform
        assert split in ['train', 'test']
        
        self.class2label = class2label
        
        self.path, self.label = list(), list()
        
        if self.split == 'train':
            self.classes = sorted(os.listdir(os.path.join(self.root, self.split)))
            self.le = {self.classes[i]: self.class2label[self.classes[i]] for i in range(len(self.classes))}
        
            for class_name in self.classes:
                class_path = os.path.join(self.root, self.split, class_name)

                for off in os.listdir(class_path):
                    if off.endswith('off'):
                        self.path.append(os.path.join(class_path, off))
                        self.label.append(self.le[class_name])
                    else:
                        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), off)
                        
        elif self.split == 'test':
            fname_list = natsorted(os.listdir(os.path.join(self.root, self.split)))
            for fname in fname_list:
                self.path.append(os.path.join(self.root, self.split, fname))
                
    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud
            
    def __getitem__(self, index):
        pcd_path = self.path[index]

        pointcloud = self.__preproc__(pcd_path)

        if self.split == 'train':
            return {'pointcloud': pointcloud, 
                'category': self.label[index]}
        
        elif self.split == 'test':
            return {'pointcloud': pointcloud}
    
    def __len__(self):
        return len(self.path)

In [174]:
### Load class2Label file
with open(LABEL_PATH, 'r') as j:
    class2label = json.load(j)

In [175]:
train_dataset = ModelNet10(root=DATA_PATH, split='train', class2label=class2label, transform = default_transforms())
test_dataset = ModelNet10(root=DATA_PATH, split='test', class2label=class2label, transform = default_transforms())

In [176]:
print('Train dataset size: ', len(train_dataset))
print('Test dataset size: ', len(test_dataset))
print('Number of classes: ', len(train_dataset.classes))
print('Sample pointcloud shape: ', train_dataset[0]['pointcloud'].size())

Train dataset size:  1000
Test dataset size:  300
Number of classes:  10
Sample pointcloud shape:  torch.Size([1024, 3])


In [208]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [178]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F


### T-net (Joing Alignment Network)
class Tnet(nn.Module):
    def __init__(self, k=3):
        super().__init__()
        self.k=k
        self.conv1 = nn.Conv1d(k,64,1)
        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k*k)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, input):
        bs = input.size(0)
        xb = F.relu(self.bn1(self.conv1(input)))
        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = F.relu(self.bn3(self.conv3(xb)))
        pool = nn.MaxPool1d(xb.size(-1))(xb)
        flat = nn.Flatten(1)(pool)
        xb = F.relu(self.bn4(self.fc1(flat)))
        xb = F.relu(self.bn5(self.fc2(xb)))

        ###initialize as identity
        init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
        if xb.is_cuda:
            init=init.cuda()
        matrix = self.fc3(xb).view(-1,self.k,self.k) + init
        return matrix


class Transform(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)


        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, input):
        matrix3x3 = self.input_transform(input)
        ### batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        
        ### Max pooling for unordered point set
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        
        ### Make global descriptor
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

In [179]:
### PointNet Loss
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

In [180]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [181]:
model = PointNet()
model.to(device);

### Load a pre-trained model if it exists
model.load_state_dict(torch.load('/kaggle/input/kaggleinputmodelnet10-classification/save.pth'))

<All keys matched successfully>

In [182]:
### Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

In [191]:
### Train
from tqdm import tqdm
def train(model, train_loader, epochs=4):
    for epoch in range(epochs): 
        model.train()
        running_loss = 0.0
        for i, data in enumerate(tqdm(train_dataloader, 0)):
            inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = model(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            ### print statistics
            running_loss += loss.item()
            if i % 5 == 4:    # print every 10 mini-batches
                print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                    (epoch + 1, i + 1, len(train_dataloader), running_loss / 10))
                running_loss = 0.0

In [192]:
train(model, train_dataloader)

 16%|█▌        | 5/32 [00:38<03:16,  7.27s/it]

[Epoch: 1, Batch:    5 /   32], loss: 0.189


 31%|███▏      | 10/32 [01:10<02:18,  6.31s/it]

[Epoch: 1, Batch:   10 /   32], loss: 0.224


 47%|████▋     | 15/32 [02:07<02:50, 10.04s/it]

...

 78%|███████▊  | 25/32 [03:36<01:06,  9.57s/it]

[Epoch: 4, Batch:   25 /   32], loss: 0.117


 94%|█████████▍| 30/32 [04:08<00:13,  6.99s/it]

[Epoch: 4, Batch:   30 /   32], loss: 0.148


100%|██████████| 32/32 [04:21<00:00,  8.17s/it]


In [210]:
### Test
preds = []
with torch.no_grad():
    model.eval()
    for data in tqdm(test_dataloader):
        inputs = data['pointcloud'].to(device).float()
        outputs, _, _ = model(inputs.transpose(1,2))
        pred = outputs.to('cpu').argmax(dim = 1)
        preds.extend(pred.tolist())
        

100%|██████████| 300/300 [01:05<00:00,  4.60it/s]


In [211]:
submit = pd.read_csv("/kaggle/input/modelnet10-classification/modelnet10/sample_submit.csv")

In [215]:
submit["Label"] = preds

In [216]:
submit

,Fname,Label
0,0.off,9
1,1.off,1
2,2.off,7
3,3.off,8
4,4.off,1
...,...,...
295,295.off,8
296,296.off,5
297,297.off,4
298,298.off,9


In [217]:
submit.to_csv("./pointnet.csv", index = False)